<a href="https://colab.research.google.com/github/zmkarakas/data_science_project_pyspark-plotly/blob/main/data_science_project_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#to view this project, get the kaggle.json file from Kaggle under 'Create new API token' and upload it, then run all the remaining cells.

!pip install  --upgrade plotly
! pip install -q kaggle
from google.colab import files
files.upload()
# upload kaggle.json here

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"ziyamertkarakas","key":"19bd6f487f935baef3a94c19f5486ae3"}'}

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
2021-olympics-in-tokyo.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2021-olympics-in-tokyo.zip
replace Athletes.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
!pip install pyspark
#this downloads pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.3.2.tar.gz (281.4 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 4.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=d01afb6191560cf7c8c482f8063bbe47fb074f9b915acef392a7be564abcb4cd
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [9]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [10]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep = ',', index = False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')


/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entriesGender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [12]:
athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11085 entries, 0 to 11084
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        11085 non-null  object
 1   NOC         11085 non-null  object
 2   Discipline  11085 non-null  object
dtypes: object(3)
memory usage: 259.9+ KB


In [13]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [14]:
medals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           93 non-null     int64 
 1   Team/NOC       93 non-null     object
 2   Gold           93 non-null     int64 
 3   Silver         93 non-null     int64 
 4   Bronze         93 non-null     int64 
 5   Total          93 non-null     int64 
 6   Rank by Total  93 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 5.2+ KB


In [15]:
medals.sort_values('Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
92,86,Syrian Arab Republic,0,0,1,1,77
62,63,Bermuda,1,0,0,1,77
76,77,Bahrain,0,1,0,1,77
77,77,Saudi Arabia,0,1,0,1,77
78,77,Lithuania,0,1,0,1,77


In [16]:
import plotly.express as px

fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence =['Gold', 'Silver', 'Brown'], title='Number of Medals Per Country')
fig.show()

In [17]:
coaches.head(5)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [18]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [19]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(7)

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [20]:
fig = px.bar(pd_coaches_query, x= 'NOC', y= 'Count', color='Count', title='Number of Coaches from each country')
fig.show()
#shows how heavily coached are the teams from different countries. Japan is by far the the highest here, possibly suggesting that their coach
#specialize in certain subdisciplines regarding the discipline to be able to cater as much as possible to athletes

In [21]:
from pyspark.sql.functions import sum, col, desc

df1 = spark.sql('Select * from Coaches')

df1 = df1.groupBy("Discipline").count().sort(desc("count"))

df1.show(20)

#Apparently we have Basketball with highest amount of coaches, close secnd is Artistic Swimming, then Football. Other disciplines have
#considerably less coaches, this may suggest inherently that sports with large teams require more coaching.

+-----------------+-----+
|       Discipline|count|
+-----------------+-----+
|       Basketball|   74|
|Artistic Swimming|   69|
|         Football|   59|
|       Volleyball|   45|
|           Hockey|   45|
|         Handball|   29|
|Baseball/Softball|   26|
|     Rugby Sevens|   25|
|       Water Polo|   22|
+-----------------+-----+



In [22]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show(20)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [23]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').show(20)

+-----------+-------------------+-----+
|        NOC|         Discipline|Count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [24]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').toPandas()

In [25]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline')
fig.show()

In [26]:
athletes['Discipline'].value_counts().head(10)

Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: Discipline, dtype: int64

In [27]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query.Discipline.value_counts()
#Choosing the first 3 disciplines here from the previous graph as there were too many.

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [28]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline (Athletics, Swimming, Football)')
fig.show()

In [29]:
entriesGender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [30]:
len(entriesGender)

46

In [31]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'}, {'type':'domain'}]] * 23
fig = make_subplots(rows=23, cols=2, 
                    subplot_titles=entriesGender['Discipline'], specs=specs)
sub_figs= []
for index, row in entriesGender.iterrows():
  sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'],
                                                      row['Male']])
  sub_figs.append(sub_fig)

k = 0
for i in range(1,24):
  for j in range(1,3):
    fig.add_trace(sub_figs[k], i , j)
    k += 1

fig.update_layout(showlegend=False, height=9000, width=600, title_text="Distribution of gender amongst discipline")
fig.update_traces(textposition='inside', textinfo='label+percent',hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()

#From this chart formation, we can see that Artistic Swimming and Rhythmic Gymnastics has no Male competitiors, and Disciplines such as
#Wrestling, Cycling, Boxing are more aggregated towards Males. And some disciplines are 50/50.

In [32]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [33]:
#Shows how many disciplines each country is participating in

spark.read.csv('Teams.csv', header=True, inferSchema=True).createOrReplaceTempView('Teams')

pd_teams_query = spark.sql('Select NOC, Count(Discipline) as count from Teams group by NOC')


pd_teams_query.show()

+--------------------+-----+
|                 NOC|count|
+--------------------+-----+
|       Côte d'Ivoire|    1|
|      Chinese Taipei|    7|
|Islamic Republic ...|    3|
|              Sweden|    6|
|   Republic of Korea|   19|
|            Malaysia|    1|
|           Singapore|    1|
|                Fiji|    2|
|              Turkey|    4|
|             Germany|   36|
|              France|   33|
|              Greece|    6|
|            Slovakia|    1|
|    Hong Kong, China|    9|
|           Argentina|   10|
|             Belgium|   10|
|              Angola|    1|
|       Great Britain|   28|
|             Ecuador|    1|
|               Qatar|    1|
+--------------------+-----+
only showing top 20 rows



In [50]:
pandasTeams = pd_teams_query.toPandas()

pandasTeams.value_counts()

fig = px.bar(pandasTeams, x= 'NOC', y= 'count', color='count', title='Number of different Disciplines per Country')

fig.update_layout(xaxis_tickfont_size=8) #downsizing the tickfont size
fig.show()

#Interesting find: Japan is actually participating in more disciplines than even the USA, even though Gold Medals are higher in USA.
#Could this be because Japan is the host country of 2021 olympics?